# Fillter

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
filepath = '../../data/raw/df.xlsx'
df = pd.read_excel(filepath)

# 5年ごとにする
# df = df[df['years'] % 5 == 0]

# 遠すぎる島を除外
df = df[df['distance_m'] <= 1991]

# 人口変動率を追加
df['population_change_rate'] = df.groupby('island_ids')['populations'].pct_change(fill_method=None) * 100

df['control_group'] = 1 - df['treatment_group']

# 経過年数を追加
df['time_since_intervention'] = np.where(df['treatment_group'] == 1, df['years'] - df['year_bridges_opened'], 0)

# df.dropna(subset='population_change_rate', inplace=True)

# 同じisland_id内でafter_treatment == 0がないものを除外
# already_treated = df.groupby('island_ids').filter(lambda x: (x['after_treated'] == 0).sum() == 0)
# df = df[~df['island_ids'].isin(already_treated['island_ids'])]

# 観測期間が短い島を除外
# missing_1975_islands = df[df['years'] == 1975]['island_ids'].unique()
# all_islands = df['island_ids'].unique()
# islands_without_1975 = [island for island in all_islands if island not in missing_1975_islands]
# print(islands_without_1975)
# df = df[~df['island_ids'].isin(islands_without_1975)]

In [3]:
df.drop(columns=['region_names', 'prefecture_names', 'island_names'], inplace=True)
df = df[['island_ids'] + [col for col in df.columns if col != 'island_ids']]

In [4]:
df

,island_ids,years,region_codes,prefecture_codes,populations,treatment_group,treatment_group_mainland,year_bridges_opened,after_treated,year_connect_mainland,...,income,area_km2,distance_m,elementary_school,junior_high_school,high_school,hospital,population_change_rate,control_group,time_since_intervention
119,67.0,1970.0,32202,32.0,NaN,1.0,1.0,1999.0,0.0,1999.0,...,NaN,0.28,615.0,NaN,NaN,NaN,NaN,NaN,0.0,-29.0
120,67.0,1975.0,32202,32.0,NaN,1.0,1.0,1999.0,0.0,1999.0,...,NaN,0.28,615.0,NaN,NaN,NaN,NaN,NaN,0.0,-24.0
121,67.0,1980.0,32202,32.0,NaN,1.0,1.0,1999.0,0.0,1999.0,...,NaN,0.28,615.0,NaN,NaN,NaN,NaN,NaN,0.0,-19.0
122,67.0,1985.0,32202,32.0,NaN,1.0,1.0,1999.0,0.0,1999.0,...,4.243674e+07,0.28,615.0,NaN,NaN,NaN,NaN,NaN,0.0,-14.0
123,67.0,1990.0,32202,32.0,NaN,1.0,1.0,1999.0,0.0,1999.0,...,4.786833e+07,0.28,615.0,NaN,NaN,NaN,NaN,NaN,0.0,-9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3517,226.0,2000.0,39424,39.0,398.0,1.0,1.0,1957.0,1.0,1957.0,...,5.113370e+02,0.56,110.0,NaN,NaN,NaN,NaN,-4.784689,0.0,43.0
3518,226.0,2005.0,39424,39.0,365.0,1.0,1.0,1957.0,1.0,1957.0,...,4.533435e+02,0.56,110.0,NaN,NaN,NaN,NaN,-8.291457,0.0,48.0
3519,226.0,2010.0,39424,39.0,343.0,1.0,1.0,1957.0,1.0,1957.0,...,3.903833e+02,0.56,110.0,NaN,NaN,NaN,NaN,-6.027397,0.0,53.0
3520,226.0,2015.0,39424,39.0,284.0,1.0,1.0,1957.0,1.0,1957.0,...,4.064721e+02,0.56,110.0,NaN,NaN,NaN,NaN,-17.201166,0.0,58.0


In [17]:
# Cohort
cohort_list = df['year_bridges_opened'].unique()
cohort_list.sort()
cohort_list

array([   0, 1951, 1961, 1967, 1972, 1973, 1975, 1976, 1979, 1983, 1987,
       1988, 1989, 1991, 1992, 1995, 1996, 1999, 2000, 2004, 2011, 2015,
       2016])

In [19]:
def create_cohort(cohort_year):
    df.loc[df['year_bridges_opened'] == cohort_year, f'cohort_{cohort_year}'] = 1
    df.loc[df['year_bridges_opened'] != cohort_year, f'cohort_{cohort_year}'] = 0
    df.drop(columns='cohort_0')
    df.loc[df[f'cohort_{cohort_year}'] == 1, f'years_cohort_{cohort_year}'] = df['passage_of_year'] * df[f'cohort_{cohort_year}']
    df.loc[df[f'cohort_{cohort_year}'] == 0, f'years_cohort_{cohort_year}'] = 0

In [21]:
for cohort_year in cohort_list:
    create_cohort(cohort_year)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1900 entries, 119 to 3521
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   island_names              1900 non-null   object 
 1   years                     1900 non-null   float64
 2   island_ids                1900 non-null   float64
 3   region_codes              1900 non-null   object 
 4   region_names              1900 non-null   object 
 5   prefecture_codes          1900 non-null   float64
 6   populations               1697 non-null   float64
 7   treatment_group           1900 non-null   float64
 8   treatment_group_mainland  1900 non-null   float64
 9   year_bridges_opened       1900 non-null   float64
 10  after_treated             1900 non-null   float64
 11  year_connect_mainland     1900 non-null   float64
 12  after_treated_mainland    1900 non-null   float64
 13  income                    1401 non-null   float64
 14  prefecture_

In [5]:
df_filtered = df
export_filepath = '../../data/processed/df_filtered.xlsx'
df_filtered.to_excel(export_filepath, index=False)